In [13]:
# 先从简单的CNN开始写吧，嘶，主要是数据太少了orzzzzz
# 所以只能做分类了，完蛋，我开始写之前怎么就没想到这点呢
# 没事，3个类，33个数据一类，勉勉强强够了，大不了我自己再洗一年的数据

import pandas as pd
from utils import Accumulator
from torch import nn
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler  # 归一化
from sklearn.model_selection import KFold         # k折交叉验
from sklearn.decomposition import PCA             # pca

def rank2class(df):
    if df['LgRk'] <= 6:
        return 0
    elif 6 < df['LgRk'] <= 13:
        return 1
    else:
        return 2

In [14]:
class myDataset():
    def __init__(self, renorm=False, pca=False, **kargs):
        '''记得如果使用pca需要添加参数pca_dim '''
        self.player = pd.read_csv('DLdata\outfielder_combined_from_raw_fillna.csv', header=0, encoding='utf-8')
        self.player = self.player.drop(['Player', 'Pos', 'Comp', 'Age', 'Born'], axis=1)
        scaler = StandardScaler()  # z score 归一化
        for col in self.player.columns:
            if col in ('Squad', "Season"): continue
            self.player[[col]] = scaler.fit_transform(self.player[[col]])


        self.league = pd.read_excel('DLdata/SquadPerformance2021.xlsx', sheet_name='Sheet1')
        self.league['tier'] = self.league.apply(rank2class, axis=1)
        self.league = self.league[['Squad', 'Season', 'tier']]  

        if pca:
            squad = self.player['Squad']
            season = self.player['Season']
            values = self.player.drop(['Squad', 'Season'], axis=1).values
            comp_dim = kargs['pca_dim']
            pca = PCA(n_components=comp_dim)   # 15
            values = pca.fit_transform(values)
            # print(sum(pca.explained_variance_ratio_))
            self.player = pd.DataFrame(values)
            self.player['Squad'] = squad
            self.player["Season"] = season

        self.X = []
        self.y = []
        for id in self.league.index:
            squad = self.league.at[id, 'Squad']
            season = self.league.at[id, 'Season']
            if season == 2021: continue  # 发现github的数据在2021年仅仅只有前几场比赛的，所以不得不舍弃了orzzz
            self.X.append(self.Squad2Player(squad, season).reshape((1, -1)))  # reshape (207,)  -> (1, 270)
            self.y.append(self.league.at[id, 'tier'])

        self.X = np.concatenate(self.X, axis=0)
        self.y = np.array(self.y)

        #if pca:
        #    pca = PCA(n_components=15)   # 15 is best
        #    pca.fit(self.X)
        #    self.X = pca.transform(self.X)
            #print(sum(pca.explained_variance_ratio_))  # 测试此时的总解释方差达到 99%

    def Squad2Player(self, squad, season):
        "输入squad的名字，输出球队球员的数据"
        season = str(season) + '-' + str(int(season) + 1)
        re = self.player[self.player['Squad'] == squad]
        #print(re.columns)
        re = re[re['Season'] == season].drop(['Squad', 'Season'], axis=1)
        re = re.values  # shape [nplayers, features]
        # print(re.shape) [20, 151]

        mean = np.mean(re, axis=0)
        max = np.max(re, axis=0)
        min = np.min(re, axis=0)
        re = np.concatenate((mean, max, min), axis=0)
        #print(re.shape) # 151*3
        return re      

    # 使用K折交叉验证
    def DataIter(self, K=10, shuffle=True):
        KF = KFold(n_splits=K, shuffle=shuffle)  
        for train_index, test_index in KF.split(self.X):
            #print("TRAIN", train_index, "TEST", test_index)
            X_train, X_test = self.X[train_index], self.X[test_index]
            Y_train, Y_test = self.y[train_index], self.y[test_index]
            yield X_train, Y_train, X_test, Y_test
    

mydataset = myDataset() #pca=1, pca_dim=15
print(mydataset.X.shape)

(392, 453)


In [15]:
# 接下来进行实验，K折交叉验证的实验
EXPERIMENT = 5
K = 10

In [17]:
# SVM
from sklearn import svm
mydataset = myDataset(renorm=0)
exp_stats_train = []
exp_stats_valid = []

for exp in range(EXPERIMENT):
    for X_train, Y_train, X_test, Y_test in mydataset.DataIter(K=K):
        model = svm.SVC(
            C=1.0,
            kernel='linear',  # linear效果最好
            gamma='scale',
            decision_function_shape='ovo'
        )
        model.fit(X_train, Y_train.ravel())    #ravel函数在降维时默认是行序优先
        #利用classifier.score（）分别计算训练集和测试集的准确率。
        train_score = model.score(X_train, Y_train)
        print("训练集：",train_score, end='  ')
        exp_stats_train.append(train_score)
        test_score = model.score(X_test, Y_test)
        print("测试集：",test_score)
        exp_stats_valid.append(test_score)


exp_stats_train = np.array(exp_stats_train)
exp_stats_valid = np.array(exp_stats_valid)
print(f'训练集acc: mean = {exp_stats_train.mean():.3f}, std = {exp_stats_train.std():.3f}')
print(f'估计泛化误差：验证集acc: mean = {exp_stats_valid.mean():.3f}, std = {exp_stats_valid.std():.3f}')


训练集： 1.0  测试集： 0.825
训练集： 1.0  测试集： 0.725
训练集： 1.0  测试集： 0.6923076923076923
训练集： 1.0  测试集： 0.6923076923076923
训练集： 1.0  测试集： 0.7435897435897436
训练集： 1.0  测试集： 0.6410256410256411
训练集： 1.0  测试集： 0.7692307692307693
训练集： 1.0  测试集： 0.6666666666666666
训练集： 1.0  测试集： 0.8205128205128205
训练集： 1.0  测试集： 0.6666666666666666
训练集： 1.0  测试集： 0.675
训练集： 1.0  测试集： 0.625
训练集： 1.0  测试集： 0.6153846153846154
训练集： 1.0  测试集： 0.7435897435897436
训练集： 1.0  测试集： 0.7435897435897436
训练集： 1.0  测试集： 0.717948717948718
训练集： 1.0  测试集： 0.6923076923076923
训练集： 1.0  测试集： 0.7435897435897436
训练集： 1.0  测试集： 0.7692307692307693
训练集： 1.0  测试集： 0.6410256410256411
训练集： 1.0  测试集： 0.7
训练集： 1.0  测试集： 0.675
训练集： 1.0  测试集： 0.7692307692307693
训练集： 1.0  测试集： 0.7692307692307693
训练集： 1.0  测试集： 0.7435897435897436
训练集： 1.0  测试集： 0.6666666666666666
训练集： 1.0  测试集： 0.5641025641025641
训练集： 1.0  测试集： 0.7692307692307693
训练集： 1.0  测试集： 0.7692307692307693
训练集： 1.0  测试集： 0.7692307692307693
训练集： 1.0  测试集： 0.75
训练集： 1.0  测试集： 0.775
训练集： 1.0  测试集： 0.589

In [18]:
# SVM + pca
# 感觉要被baseline薄纱了
mydataset = myDataset(pca=1, pca_dim=15)
print(mydataset.X.shape)
exp_stats_train = []
exp_stats_valid = []
for exp in range(EXPERIMENT):
    for X_train, Y_train, X_test, Y_test in mydataset.DataIter(K=K):
        model = svm.SVC(
            C=1.0,
            kernel='linear',
            gamma='scale',
            decision_function_shape='ovo'
        )
        model.fit(X_train, Y_train.ravel())    #ravel函数在降维时默认是行序优先
        #利用classifier.score（）分别计算训练集和测试集的准确率。
        train_score = model.score(X_train, Y_train)
        print("训练集：",train_score, end='  ')
        exp_stats_train.append(train_score)
        test_score = model.score(X_test, Y_test)
        print("测试集：",test_score)
        exp_stats_valid.append(test_score)


exp_stats_train = np.array(exp_stats_train)
exp_stats_valid = np.array(exp_stats_valid)
print(f'训练集acc: mean = {exp_stats_train.mean():.3f}, std = {exp_stats_train.std():.3f}')
print(f'估计泛化误差：验证集acc: mean = {exp_stats_valid.mean():.3f}, std = {exp_stats_valid.std():.3f}')

(392, 45)
训练集： 0.9034090909090909  测试集： 0.8
训练集： 0.9147727272727273  测试集： 0.75
训练集： 0.9206798866855525  测试集： 0.7692307692307693
训练集： 0.9291784702549575  测试集： 0.7435897435897436
训练集： 0.8838526912181303  测试集： 0.7435897435897436
训练集： 0.9178470254957507  测试集： 0.6666666666666666
训练集： 0.9036827195467422  测试集： 0.7948717948717948
训练集： 0.9036827195467422  测试集： 0.8461538461538461
训练集： 0.9121813031161473  测试集： 0.7692307692307693
训练集： 0.9036827195467422  测试集： 0.8717948717948718
训练集： 0.9176136363636364  测试集： 0.825
训练集： 0.8948863636363636  测试集： 0.85
训练集： 0.9065155807365439  测试集： 0.6666666666666666
训练集： 0.9178470254957507  测试集： 0.7948717948717948
训练集： 0.9093484419263456  测试集： 0.8461538461538461
训练集： 0.9065155807365439  测试集： 0.8717948717948718
训练集： 0.8951841359773371  测试集： 0.717948717948718
训练集： 0.9093484419263456  测试集： 0.6923076923076923
训练集： 0.9036827195467422  测试集： 0.7435897435897436
训练集： 0.9121813031161473  测试集： 0.7435897435897436
训练集： 0.9090909090909091  测试集： 0.8
训练集： 0.8977272727272727  测试集： 0.9

In [20]:
# RF
from sklearn.ensemble import RandomForestClassifier
mydataset = myDataset(renorm=0)
exp_stats_train = []
exp_stats_valid = []

for exp in range(EXPERIMENT):
    for X_train, Y_train, X_test, Y_test in mydataset.DataIter(K=K):
        model = RandomForestClassifier(
            n_estimators=50,
            criterion='entropy',
            max_depth=5
        )
        model.fit(X_train, Y_train.ravel())    #ravel函数在降维时默认是行序优先
        #利用classifier.score（）分别计算训练集和测试集的准确率。
        train_score = model.score(X_train, Y_train)
        print("训练集：",train_score, end='  ')
        exp_stats_train.append(train_score)
        test_score = model.score(X_test, Y_test)
        print("测试集：",test_score)
        exp_stats_valid.append(test_score)


exp_stats_train = np.array(exp_stats_train)
exp_stats_valid = np.array(exp_stats_valid)
print(f'训练集acc: mean = {exp_stats_train.mean():.3f}, std = {exp_stats_train.std():.3f}')
print(f'估计泛化误差：验证集acc: mean = {exp_stats_valid.mean():.3f}, std = {exp_stats_valid.std():.3f}')


训练集： 0.9801136363636364  测试集： 0.775
训练集： 0.9772727272727273  测试集： 0.8
训练集： 0.9660056657223796  测试集： 0.8461538461538461
训练集： 0.9830028328611898  测试集： 0.8974358974358975
训练集： 0.9801699716713881  测试集： 0.6666666666666666
训练集： 0.9773371104815864  测试集： 0.7435897435897436
训练集： 0.9660056657223796  测试集： 0.8974358974358975
训练集： 0.9830028328611898  测试集： 0.6923076923076923
训练集： 0.9773371104815864  测试集： 0.8205128205128205
训练集： 0.9745042492917847  测试集： 0.8461538461538461
训练集： 0.9772727272727273  测试集： 0.85
训练集： 0.9829545454545454  测试集： 0.7
训练集： 0.9716713881019831  测试集： 0.7692307692307693
训练集： 0.9773371104815864  测试集： 0.8205128205128205
训练集： 0.9660056657223796  测试集： 0.8461538461538461
训练集： 0.9745042492917847  测试集： 0.8205128205128205
训练集： 0.9745042492917847  测试集： 0.7948717948717948
训练集： 0.9858356940509915  测试集： 0.8461538461538461
训练集： 0.9801699716713881  测试集： 0.8974358974358975
训练集： 0.9858356940509915  测试集： 0.7692307692307693
训练集： 0.9772727272727273  测试集： 0.775
训练集： 0.9630681818181818  测试集： 0.825
训练集： 

In [21]:

# RF + pca
# 感觉要被baseline薄纱了
mydataset = myDataset(pca=1, pca_dim=15)
print(mydataset.X.shape)
exp_stats_train = []
exp_stats_valid = []
for exp in range(EXPERIMENT):
    for X_train, Y_train, X_test, Y_test in mydataset.DataIter(K=K):
        model = RandomForestClassifier(
            n_estimators=50,
            criterion='entropy',
            max_depth=5
        )
        model.fit(X_train, Y_train.ravel())    #ravel函数在降维时默认是行序优先
        #利用classifier.score（）分别计算训练集和测试集的准确率。
        train_score = model.score(X_train, Y_train)
        print("训练集：",train_score, end='  ')
        exp_stats_train.append(train_score)
        test_score = model.score(X_test, Y_test)
        print("测试集：",test_score)
        exp_stats_valid.append(test_score)


exp_stats_train = np.array(exp_stats_train)
exp_stats_valid = np.array(exp_stats_valid)
print(f'训练集acc: mean = {exp_stats_train.mean():.3f}, std = {exp_stats_train.std():.3f}')
print(f'估计泛化误差：验证集acc: mean = {exp_stats_valid.mean():.3f}, std = {exp_stats_valid.std():.3f}')

(392, 45)
训练集： 0.9630681818181818  测试集： 0.6
训练集： 0.9630681818181818  测试集： 0.725
训练集： 0.9575070821529745  测试集： 0.7435897435897436
训练集： 0.9575070821529745  测试集： 0.8205128205128205
训练集： 0.9518413597733711  测试集： 0.8717948717948718
训练集： 0.9490084985835694  测试集： 0.8205128205128205
训练集： 0.9546742209631728  测试集： 0.8717948717948718
训练集： 0.9546742209631728  测试集： 0.8717948717948718
训练集： 0.9660056657223796  测试集： 0.7948717948717948
训练集： 0.9603399433427762  测试集： 0.8205128205128205
训练集： 0.9630681818181818  测试集： 0.825
训练集： 0.9573863636363636  测试集： 0.725
训练集： 0.9660056657223796  测试集： 0.7692307692307693
训练集： 0.9546742209631728  测试集： 0.7692307692307693
训练集： 0.9660056657223796  测试集： 0.8205128205128205
训练集： 0.9546742209631728  测试集： 0.8717948717948718
训练集： 0.9745042492917847  测试集： 0.8717948717948718
训练集： 0.9575070821529745  测试集： 0.7435897435897436
训练集： 0.9631728045325779  测试集： 0.7948717948717948
训练集： 0.9518413597733711  测试集： 0.7948717948717948
训练集： 0.9602272727272727  测试集： 0.725
训练集： 0.9630681818181818  测试集